# Run Models on 2019 Data Joined with Median Home Value and Gross Rent
#### 10/23/2020
---

## Load and Process Dataset
### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import wandb
import warnings
warnings.filterwarnings('ignore')

In [2]:
wandb.init(project="2019_newvar_home-value_gross-rent")

wandb: Currently logged in as: apraturu (use `wandb login --relogin` to force relogin)


### Read in `adult.csv` data

In [3]:
df = pd.read_csv('../final_data/2019_new-var_home-value_gross-rent.csv').sample(frac=1)
df.head()

,Geographic-division,region,state,RT,person-weight,age,citizenship-status,class-worker,education,marital-status,...,sex,extra-income,hours-per-week,field-of-degree,place-of-birth,income,stem-degree,full PUMA,med_home_value,med_gross_rent
1148382,East North Central,Midwest,39,P,1041,32,Born-US,Private-prof,bachelor,Never-married,...,male,0.0,40.0,Mechanical Engineering,Ohio/OH,20000.0,Yes,3904105,294800,1175
1667054,East North Central,Midwest,55,P,100,50,Born-US,Private-prof,HSgrad,Married,...,male,0.0,40.0,NaN,Wisconsin/WI,60000.0,NaN,5501601,151800,738
978377,Mountain,West,35,P,134,58,Born-US,Private-prof,associate,Married,...,male,0.0,60.0,NaN,Texas/TX,177000.0,NaN,3500400,111300,760
926870,Middle Atlantic,Northeast,34,P,302,38,naturalized-citizen,Private-prof,bachelor,Never-married,...,female,0.0,40.0,Commercial Art And Graphic Design,Ecuador,90000.0,No,3400302,419400,1449
677579,West South Central,South,22,P,105,44,Born-US,Private-prof,grade11,Never-married,...,female,0.0,40.0,NaN,Louisiana/LA,14500.0,NaN,2202402,318300,1071


In [4]:
df.loc[df['income'] <= 50000, 'income_adjusted'] = '<=50K'
df.loc[df['income'] > 50000, 'income_adjusted'] = '>50K' 

df = df.drop(columns=['income']).rename(columns={'income_adjusted': 'income'})#.dropna()

df['med_home_value'] = pd.to_numeric(df['med_home_value'],errors='coerce')

df['field-of-degree'] = df['field-of-degree'].fillna('Unknown')
df['stem-degree'] = df['stem-degree'].fillna('Unknown')

# df = df.dropna()

df.head()

,Geographic-division,region,state,RT,person-weight,age,citizenship-status,class-worker,education,marital-status,...,sex,extra-income,hours-per-week,field-of-degree,place-of-birth,stem-degree,full PUMA,med_home_value,med_gross_rent,income
1148382,East North Central,Midwest,39,P,1041,32,Born-US,Private-prof,bachelor,Never-married,...,male,0.0,40.0,Mechanical Engineering,Ohio/OH,Yes,3904105,294800.0,1175,<=50K
1667054,East North Central,Midwest,55,P,100,50,Born-US,Private-prof,HSgrad,Married,...,male,0.0,40.0,Unknown,Wisconsin/WI,Unknown,5501601,151800.0,738,>50K
978377,Mountain,West,35,P,134,58,Born-US,Private-prof,associate,Married,...,male,0.0,60.0,Unknown,Texas/TX,Unknown,3500400,111300.0,760,>50K
926870,Middle Atlantic,Northeast,34,P,302,38,naturalized-citizen,Private-prof,bachelor,Never-married,...,female,0.0,40.0,Commercial Art And Graphic Design,Ecuador,No,3400302,419400.0,1449,>50K
677579,West South Central,South,22,P,105,44,Born-US,Private-prof,grade11,Never-married,...,female,0.0,40.0,Unknown,Louisiana/LA,Unknown,2202402,318300.0,1071,<=50K


In [6]:
df = df.dropna()

df.isnull().sum()

Geographic-division    0
region                 0
state                  0
RT                     0
person-weight          0
age                    0
citizenship-status     0
class-worker           0
education              0
marital-status         0
occupation             0
relationship           0
race                   0
sex                    0
extra-income           0
hours-per-week         0
field-of-degree        0
place-of-birth         0
stem-degree            0
full PUMA              0
med_home_value         0
med_gross_rent         0
income                 0
dtype: int64

In [7]:
X = df.drop(columns=['income'], axis = 1)
y = df[['income']].to_numpy()

In [8]:
labels = np.unique(y)
features = list(X.columns)

### Pre-process categorical variables

In [9]:
cat_var = ['Geographic-division', 'region', 'state', 'RT', 'citizenship-status', 'class-worker', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'field-of-degree', 'place-of-birth', 'stem-degree']
for f in cat_var:
    enc = preprocessing.LabelEncoder()
    X[f] = enc.fit_transform(X[f])

scaler = preprocessing.StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

### Split data into test and training

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(X_test)

333901

## Train Models and Establish Baseline Scores

### Logistic Regression

In [12]:
# Train model, get predictions
model = LogisticRegression()
model.fit(X, y)
y_pred = model.predict(X_test)
y_probas = model.predict_proba(X_test)
importances = model.coef_
indices = np.argsort(importances)[::-1]

In [13]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1335603 entries, 1248682 to 892259
Data columns (total 22 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Geographic-division  1335603 non-null  float64
 1   region               1335603 non-null  float64
 2   state                1335603 non-null  float64
 3   RT                   1335603 non-null  float64
 4   person-weight        1335603 non-null  float64
 5   age                  1335603 non-null  float64
 6   citizenship-status   1335603 non-null  float64
 7   class-worker         1335603 non-null  float64
 8   education            1335603 non-null  float64
 9   marital-status       1335603 non-null  float64
 10  occupation           1335603 non-null  float64
 11  relationship         1335603 non-null  float64
 12  race                 1335603 non-null  float64
 13  sex                  1335603 non-null  float64
 14  extra-income         1335603 non-null  float6

In [14]:
# Visualize model performance
wandb.sklearn.plot_classifier(
    model, X_train, X_test, y_train, y_test, y_pred, y_probas, labels,
    is_binary=True, model_name='LogisticRegression', feature_names=features)

wandb: 
wandb: Plotting LogisticRegression.
wandb: Logged feature importances.
wandb: Logged learning curve.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: Logged roc curve.
wandb: Logged precision recall curve.


In [15]:
print(classification_report(y_test, y_pred, target_names=labels))

              precision    recall  f1-score   support

       <=50K       0.78      0.84      0.81    202206
        >50K       0.73      0.64      0.68    131695

    accuracy                           0.76    333901
   macro avg       0.75      0.74      0.75    333901
weighted avg       0.76      0.76      0.76    333901

